In [ ]:
import json
import os
import shutil
from tqdm import tqdm
import yaml

In [ ]:
CATEGORY = yaml.safe_load(open("STEP_0_define_category_sub_category.yaml", "r"))['category']
SUB_CATEGORY = yaml.safe_load(open("STEP_0_define_category_sub_category.yaml", "r"))['sub_category']
print(f"Category: {CATEGORY}")
print(f"Sub Category: {SUB_CATEGORY}")

In [ ]:
REPO_DIR = '/Users/stevie/repos/language_app'
assert(os.path.exists(REPO_DIR))

DEST_DIR = os.path.join('/Users/stevie/repos/language-app-frontend/assets/category_data', f"{CATEGORY}_{SUB_CATEGORY}")
os.makedirs(DEST_DIR, exist_ok=True)

In [ ]:
json_path = os.path.join(REPO_DIR, f'category_data/step_2/{CATEGORY}_{SUB_CATEGORY}.json')
data = json.load(open(json_path))

In [ ]:
path_rel = data[0]['combined_audio_file']
print(path_rel)
assert(os.path.exists(os.path.join(REPO_DIR, path_rel)))
# print(os.path.join(DEST_DIR, path_rel))

In [ ]:
for sample_i, sample_obj in tqdm(enumerate(data), total=len(data)):
    # combined audio file
    path = os.path.join(REPO_DIR, sample_obj['combined_audio_file'])
    assert(os.path.exists(path))
    rel_new_path = f'./audio/combined/{os.path.basename(path)}'
    new_path = os.path.join(DEST_DIR, rel_new_path)
    os.makedirs(os.path.dirname(new_path), exist_ok=True)
    if not os.path.exists(new_path):
        shutil.copyfile(path, new_path)
    path_str = f'require("{rel_new_path}"):::'
    data[sample_i]['combined_audio_file'] = path_str

    # individual audio files
    for language in 'english', 'italian':
        path = os.path.join(REPO_DIR, sample_obj[language]['audio_file'])
        assert(os.path.exists(path))
        rel_new_path = f'./audio/individual/{os.path.basename(path)}'
        new_path = os.path.join(DEST_DIR, rel_new_path)
        os.makedirs(os.path.dirname(new_path), exist_ok=True)
        if not os.path.exists(new_path):
            shutil.copyfile(path, new_path)
        path_str = f'require("{rel_new_path}"):::'
        data[sample_i][language]['audio_file'] = path_str

In [ ]:
REPO_DIR

In [ ]:
js_path = os.path.join(REPO_DIR, f"category_data/step_2/{CATEGORY}_{SUB_CATEGORY}.js")
json.dump(data, open(js_path, 'w'), indent=2, ensure_ascii=False)
text = open(js_path).read()
text = text.replace('\\"', '"')
text = f"samples = {text}"
text = text.replace('"require', 'require').replace(':::"', '')
text += ';\nexport default samples;'
open(js_path, 'w').write(text)

In [ ]:
old_js_path = os.path.join(REPO_DIR, f"category_data/step_2/{CATEGORY}_{SUB_CATEGORY}.js")
new_js_path = os.path.join(DEST_DIR, 'data.js')
shutil.copyfile(old_js_path, new_js_path)

old_json_path = os.path.join(REPO_DIR, f"category_data/step_2/{CATEGORY}_{SUB_CATEGORY}.json")
new_json_path = os.path.join(DEST_DIR, 'data.json')
shutil.copyfile(old_json_path, new_json_path)